# Getting Started with BlazingSQL

In this notebook, we will cover: 
- How to set up [BlazingSQL](https://blazingsql.com) and the [RAPIDS AI](https://rapids.ai/) suite.
- How to read and query csv files with cuDF and BlazingSQL.
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=bsql-quick-start-guide&dt=bsql-quick-start-guide)

## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [1]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/colab_env.py
!python colab_env.py 



***********************************
GPU = b'Tesla T4'
Woo! You got the right kind of GPU!
***********************************




## Installs 
The cell below pulls our Google Colab install script from the `bsql-demos` repo then runs it. The script first installs miniconda, then uses miniconda to install BlazingSQL and RAPIDS AI. This takes a few minutes to run. 

In [ ]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/bsql-colab.sh 
!bash bsql-colab.sh

import sys, os, time
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])

import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) 

## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.


In [1]:
import cudf
from blazingsql import BlazingContext
# start up BlazingSQL
bc = BlazingContext()

BlazingContext ready


## Read CSV
First we need to download a CSV file. Then we use cuDF to read the CSV file, which gives us a GPU DataFrame (GDF). To learn more about the GDF and how it enables end to end workloads on rapids, check out our [blog post](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240).

In [2]:
#Download the test CSV
!wget 'https://s3.amazonaws.com/blazingsql-colab/Music.csv'

--2020-01-23 02:59:55--  https://s3.amazonaws.com/blazingsql-colab/Music.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.0.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10473 (10K) [text/csv]
Saving to: ‘Music.csv’

Music.csv           100%[===================>]  10.23K  --.-KB/s    in 0s      

2020-01-23 02:59:55 (190 MB/s) - ‘Music.csv’ saved [10473/10473]



In [3]:
# like pandas, cudf can simply read the csv
gdf = cudf.read_csv('Music.csv')

# let's see how it looks
gdf.head()

,ARTIST,RATING,YEAR,LOCATION,FESTIVAL_SET
0,Arcade Fire,10.0,2018.0,Las Vegas,1.0
1,Justice,10.0,2018.0,Las Vegas,1.0
2,Florence and The Machine,10.0,2018.0,Las Vegas,1.0
3,Odesza,10.0,2018.0,Indio,1.0
4,Bon Iver,10.0,2017.0,Indio,1.0


## Create a Table
Now we just need to create a table. 

In [4]:
bc.create_table('music', gdf, header=0)

## Query a Table
That's it! Now when you can write a SQL query the data will get processed on the GPU with BlazingSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [5]:
# query 10 events with a rating of at least 7
gdf = bc.sql('select ARTIST, RATING, LOCATION from music where RATING >= 7 limit 10')

# display GDF (just like pandas)
gdf

,ARTIST,RATING,LOCATION
0,Arcade Fire,10.0,Las Vegas
1,Justice,10.0,Las Vegas
2,Florence and The Machine,10.0,Las Vegas
3,Odesza,10.0,Indio
4,Bon Iver,10.0,Indio
5,LA Philharmonic + Sigur Ros,10.0,LA
6,Sigur Ros,10.0,Malmo
7,Arcade Fire,10.0,Indio
8,Escort,9.0,San Francisco
9,Phoenix,9.0,Berkeley


# You're Ready to Rock
And... thats it! You are now live with BlazingSQL.


Check out our [docs](https://docs.blazingdb.com) to get fancy or to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/).